In [15]:
import tensorflow as tf
import time

In [16]:
tf.__version__

'2.18.0'

In [17]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield(sample_idx, )
            
    def __new__(cls, num_samples=3):
       return tf.data.Dataset.from_generator(
           cls.read_files_in_batches,
           output_signature=tf.TensorSpec(shape=(1,),dtype=tf.int64),
           args=(num_samples,)
       )
        
       

In [18]:
def benchmark(dataset,num_epochs=2):
    for epoch in range(num_epochs):
        for sample in dataset:
            time.sleep(0.01)

In [19]:
# obj=FileDataset()

In [20]:
%%timeit
benchmark(FileDataset())

2025-03-03 08:18:46.076569: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-03-03 08:18:46.320984: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-03 08:18:46.434624: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-03 08:18:46.724988: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-03-03 08:18:47.252959: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


281 ms ± 20.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


2025-03-03 08:18:48.408169: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
%%timeit
benchmark(FileDataset().prefetch(1))

279 ms ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


2025-03-03 08:21:12.745156: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [23]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

272 ms ± 9.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
dataset=tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [31]:
dataset1=dataset.map(lambda x: x**2)
for d in dataset1:
    print(d.numpy())

0
1
4
9
16


In [33]:
dataset=dataset.cache()
for d in dataset.as_numpy_iterator():
    print(d)

0
1
2
3
4


In [34]:
list(dataset.as_numpy_iterator())


[0, 1, 2, 3, 4]

In [38]:
def mapped_function(s):
    tf.py_function(lambda:time.sleep(0.03),[],())
    return s

In [42]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function),5)

1.14 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [43]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function).cache(),5)

441 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
